# Pandas: Aggregating ir Reshaping DataFrames

Šiame faile pateikiami aiškūs ir nuoseklūs pavyzdžiai, kaip su `pandas`:

- agreguoti duomenis (sum, mean, count ir pan.)
- grupuoti pagal vieną ir kelis stulpelius
- dirbti su MultiIndex rezultatais
- naudoti `agg()` metodą, įskaitant kelias agregacijas ir „named aggregations“
- kurti pivot lenteles ir suprasti skirtumą tarp `pivot_table` ir `groupby`
- transformuoti duomenų formą su `melt()`

Pavyzdžiai remiasi paprastu pardavimų (sales) scenarijumi. Komentarai pateikiami techniniu, aiškinamuoju stiliumi.


In [1]:
import pandas as pd
import numpy as np

pd.__version__


'2.3.3'

## 1. Pavyzdiniai duomenys (pardavimų scenarijus)

Duomenys sukurti taip, kad būtų aiškiai matomi tipiniai analitikos pjūviai:

- data (diena)
- regionas
- kanalas (Online / Store)
- produktų kategorija
- kiekis (units)
- vieneto kaina (unit_price)
- nuolaida (discount)
- pajamos (revenue)

Geroji praktika: skaitinės reikšmės laikomos skaitiniu tipu, datos konvertuojamos į `datetime`.


In [2]:
sales = pd.DataFrame({
    "date": pd.to_datetime([
        "2025-01-02","2025-01-02","2025-01-03","2025-01-03","2025-01-04","2025-01-04",
        "2025-02-01","2025-02-01","2025-02-02","2025-02-02","2025-02-03","2025-02-03"
    ]),
    "region": ["North","North","South","South","North","South","North","South","North","South","North","South"],
    "channel": ["Online","Store","Online","Store","Store","Online","Online","Online","Store","Store","Online","Store"],
    "category": ["Bikes","Bikes","Accessories","Bikes","Accessories","Accessories","Bikes","Accessories","Bikes","Accessories","Accessories","Bikes"],
    "units": [2, 1, 5, 1, 3, 2, 1, 4, 2, 1, 6, 1],
    "unit_price": [800, 820, 25, 810, 30, 28, 790, 26, 815, 32, 27, 805],
    "discount": [0.05, 0.00, 0.10, 0.00, 0.00, 0.05, 0.00, 0.15, 0.00, 0.00, 0.10, 0.00],
})


sales.head()

,date,region,channel,category,units,unit_price,discount
0,2025-01-02,North,Online,Bikes,2,800,0.05
1,2025-01-02,North,Store,Bikes,1,820,0.00
2,2025-01-03,South,Online,Accessories,5,25,0.10
3,2025-01-03,South,Store,Bikes,1,810,0.00
4,2025-01-04,North,Store,Accessories,3,30,0.00


In [3]:
sales.dtypes

date          datetime64[ns]
region                object
channel               object
category              object
units                  int64
unit_price             int64
discount             float64
dtype: object

In [4]:
sales.describe

<bound method NDFrame.describe of          date region channel     category  units  unit_price  discount
0  2025-01-02  North  Online        Bikes      2         800      0.05
1  2025-01-02  North   Store        Bikes      1         820      0.00
2  2025-01-03  South  Online  Accessories      5          25      0.10
3  2025-01-03  South   Store        Bikes      1         810      0.00
4  2025-01-04  North   Store  Accessories      3          30      0.00
5  2025-01-04  South  Online  Accessories      2          28      0.05
6  2025-02-01  North  Online        Bikes      1         790      0.00
7  2025-02-01  South  Online  Accessories      4          26      0.15
8  2025-02-02  North   Store        Bikes      2         815      0.00
9  2025-02-02  South   Store  Accessories      1          32      0.00
10 2025-02-03  North  Online  Accessories      6          27      0.10
11 2025-02-03  South   Store        Bikes      1         805      0.00>

In [5]:
# Pajamos po nuolaidos (paprasta formulė)
sales["revenue"] = (sales["units"] * sales["unit_price"] * (1 - sales["discount"])).round(2)

sales.head()

,date,region,channel,category,units,unit_price,discount,revenue
0,2025-01-02,North,Online,Bikes,2,800,0.05,1520.0
1,2025-01-02,North,Store,Bikes,1,820,0.00,820.0
2,2025-01-03,South,Online,Accessories,5,25,0.10,112.5
3,2025-01-03,South,Store,Bikes,1,810,0.00,810.0
4,2025-01-04,North,Store,Accessories,3,30,0.00,90.0


## 2. Aggregating DataFrames

Agregavimas reiškia, kad keli įrašai sutraukiami į santrauką, dažniausiai pagal grupes arba per visą lentelę.

Dažna klaida: sumuoti arba vidurkinti stulpelius, kuriuose yra tekstinės reikšmės, ir tikėtis prasmingo rezultato.

Geroji praktika:
- aiškiai pasirinkti, kurie stulpeliai yra skaitiniai
- prieš agregavimą patikrinti `dtypes` ir trūkstamas reikšmes


In [6]:
sales.dtypes


date          datetime64[ns]
region                object
channel               object
category              object
units                  int64
unit_price             int64
discount             float64
revenue              float64
dtype: object

In [7]:
# Santrauka per visą lentelę
total_revenue = sales["revenue"].sum().round(2)
avg_units = sales["units"].mean().round(2)
orders_count = len(sales)

total_revenue, avg_units, orders_count


(6896.9, 2.42, 12)

## 3. Grouping DataFrames (groupby)

`groupby()` sukuria grupes pagal vieną ar kelis stulpelius ir leidžia skaičiuoti agregacijas.

Dažna klaida:
- tikėtis, kad `groupby()` grąžins paprastą DataFrame be papildomų veiksmų (reikia agregacijos, pvz. `sum()`, `mean()` ir pan.)
- pamiršti `reset_index()`, kai rezultato reikia kaip įprasto DataFrame


In [8]:
# Pajamos pagal regioną
rev_by_region = sales.groupby("region")["revenue"].sum()
rev_by_region


region
North    4995.8
South    1901.1
Name: revenue, dtype: float64

In [9]:
# Jei rezultato reikia kaip DataFrame
rev_by_region_df = rev_by_region.reset_index(name="total_revenue")
rev_by_region_df


,region,total_revenue
0,North,4995.8
1,South,1901.1


## 4. Grupavimas pagal kelis stulpelius (multiple columns)

Grupavimas pagal kelis stulpelius leidžia gauti detalų pjūvį, pavyzdžiui: regionas × kanalas.

Dažna klaida:
- netyčia sumaišyti skaitinius ir tekstinius stulpelius agregacijoje
- pamiršti, kad grąžinamas MultiIndex rezultatas


In [10]:
# Pajamos pagal regioną ir kanalą
rev_by_region_channel = (
    sales.groupby(["region", "channel"])["revenue"]
    .sum()
    .sort_values(ascending=False)
)
rev_by_region_channel


region  channel
North   Store      2540.0
        Online     2455.8
South   Store      1647.0
        Online      254.1
Name: revenue, dtype: float64

## 5. MultiIndex rezultatai ir jų tvarkymas

Grupavimas pagal kelis stulpelius dažnai sukuria `MultiIndex` (daugialygį indeksą).

Geroji praktika:
- naudoti `reset_index()`, kai rezultatas turi būti naudojamas toliau kaip įprasta lentelė
- naudoti `unstack()` arba `pivot_table`, kai reikia „plataus“ formato


In [11]:
# MultiIndex -> įprastas DataFrame
rev_multi_df = rev_by_region_channel.reset_index(name="total_revenue")
rev_multi_df


,region,channel,total_revenue
0,North,Store,2540.0
1,North,Online,2455.8
2,South,Store,1647.0
3,South,Online,254.1


In [13]:
# MultiIndex -> platus formatas (stulpeliai pagal kanalą)
rev_wide = rev_by_region_channel.unstack(fill_value=0)
rev_wide


channel,Online,Store
region,,
North,2455.8,2540.0
South,254.1,1647.0


## 6. `agg()` metodas

`agg()` leidžia vienu metu apskaičiuoti kelias agregacijas, taip pat skirtingas agregacijas skirtingiems stulpeliams.

Dažna klaida:
- bandyti perduoti neteisingą agregacijos pavadinimą (pvz., rašybos klaida)
- tikėtis, kad `agg()` automatiškai sukurs prasmingus stulpelių pavadinimus be aiškios struktūros


In [16]:
# Kelios agregacijos vienam stulpeliui
agg_revenue = sales.groupby("region")["revenue"].agg(["sum", "mean", "count"])
agg_revenue


,sum,mean,count
region,,,
North,4995.8,832.633333,6
South,1901.1,316.850000,6


## 7. Multiple aggregations (kelios agregacijos keliems stulpeliams)

Ši forma leidžia nurodyti, kokias agregacijas taikyti kiekvienam stulpeliui.

Geroji praktika:
- aiškiai pasirinkti metrikas (pvz., `sum` pajamoms, `mean` kainai, `sum` vienetams)
- apvalinti rezultatą, kad būtų patogiau skaityti


In [17]:
agg_multi = (
    sales.groupby(["region", "channel"])
    .agg({
        "revenue": ["sum", "mean"],
        "units": ["sum", "mean"],
        "unit_price": "mean",
        "discount": "mean",
    })
)

# Patogumui: apvalinimas
agg_multi.round(2)


revenue         units       unit_price discount
                   sum    mean   sum  mean       mean     mean
region channel                                                
North  Online   2455.8  818.60     9  3.00     539.00     0.05
       Store    2540.0  846.67     6  2.00     555.00     0.00
South  Online    254.1   84.70    11  3.67      26.33     0.10
       Store    1647.0  549.00     3  1.00     549.00     0.00

## 8. Named aggregations

„Named aggregations“ leidžia aiškiai įvardinti rezultatinius stulpelius.

Tai yra viena švariausių formų, kai reikia tvarkingo, lengvai eksportuojamo rezultato.

Dažna klaida:
- sumaišyti sintaksę: `new_name=("col", "agg")` yra privaloma struktūra


In [18]:
summary = (
    sales.groupby(["region", "channel"])
    .agg(
        total_revenue=("revenue", "sum"),
        avg_revenue=("revenue", "mean"),
        total_units=("units", "sum"),
        avg_units=("units", "mean"),
        avg_price=("unit_price", "mean"),
        avg_discount=("discount", "mean"),
        orders=("revenue", "count"),
    )
    .round(2)
    .reset_index()
)

summary


,region,channel,total_revenue,avg_revenue,total_units,avg_units,avg_price,avg_discount,orders
0,North,Online,2455.8,818.60,9,3.00,539.00,0.05,3
1,North,Store,2540.0,846.67,6,2.00,555.00,0.00,3
2,South,Online,254.1,84.70,11,3.67,26.33,0.10,3
3,South,Store,1647.0,549.00,3,1.00,549.00,0.00,3


## 9. Pivot lentelės (pivot tables)

`pivot_table()` sukuria santrauką „plataus“ formato lentele.

Dažna klaida:
- naudoti `pivot()` vietoje `pivot_table()` ir gauti klaidą dėl pasikartojančių kombinacijų
- pamiršti `fill_value`, dėl ko atsiranda `NaN`, nors logiškai norisi 0


In [19]:
pivot_rev = pd.pivot_table(
    sales,
    index="region",
    columns="channel",
    values="revenue",
    aggfunc="sum",
    fill_value=0
)

pivot_rev


channel,Online,Store
region,,
North,2455.8,2540.0
South,254.1,1647.0


## 10. Pivot table vs. groupby

Pagrindinė idėja:

- `groupby()` dažniausiai grąžina „ilgą“ formatą (arba MultiIndex), kurį galima toliau transformuoti.
- `pivot_table()` iš karto pateikia „platų“ formatą (stulpeliai pagal kategorijas).

Geroji praktika:
- naudoti `groupby()` tada, kai reikia lankstumo ir kelių skirtingų agregacijų
- naudoti `pivot_table()` tada, kai tikslas yra lentelė „ataskaitos“ formatui


In [20]:
# Tas pats rezultatas kaip pivot lentelėje, tik per groupby + unstack
groupby_like_pivot = (
    sales.groupby(["region", "channel"])["revenue"]
    .sum()
    .unstack(fill_value=0)
)

groupby_like_pivot


channel,Online,Store
region,,
North,2455.8,2540.0
South,254.1,1647.0


## 11. Reshaping su `melt()`

`melt()` naudojamas, kai reikia „plataus“ formato lentelę paversti į „ilgą“ formatą.

Tai dažna situacija, kai:
- duomenys paruošti ataskaitoms (wide)
- o analizei ar vizualizacijoms reikia tvarkingo „tidy data“ formato (long)

Dažna klaida:
- pamiršti nurodyti `id_vars`, todėl prarandamas identifikacinis kontekstas


In [21]:
# Paruošiama „plataus“ formato lentelė: pajamos pagal regioną ir kanalą
wide = pivot_rev.reset_index()
wide


channel,region,Online,Store
0,North,2455.8,2540.0
1,South,254.1,1647.0


In [22]:
# Wide -> Long
long = wide.melt(
    id_vars=["region"],
    var_name="channel",
    value_name="total_revenue"
)

long


,region,channel,total_revenue
0,North,Online,2455.8
1,South,Online,254.1
2,North,Store,2540.0
3,South,Store,1647.0


## 12. Dažnos klaidos ir gerosios praktikos (santrauka)

Dažnos klaidos:
- palikti datas kaip `object` ir vėliau gauti klaidas su `.dt`
- sumuoti tekstinius stulpelius arba maišyti metrikas be aiškaus tikslo
- pamiršti `reset_index()` po `groupby()`, kai rezultatas turi būti naudojamas kaip DataFrame
- naudoti `pivot()` vietoje `pivot_table()` ir gauti klaidą dėl dublikatų
- pamiršti `fill_value`, todėl atsiranda `NaN` vietose, kur logiškai norisi 0

Gerosios praktikos:
- prieš analizę patikrinti `dtypes` ir trūkstamas reikšmes
- naudoti `named aggregations`, kai reikia aiškių stulpelių pavadinimų
- po `groupby()` dažnai naudoti `reset_index()` aiškiam rezultatui
- rinktis `groupby()` lankstumui, `pivot_table()` ataskaitos formatui
- naudoti `melt()`, kai reikia „tidy“ formato analizei ar vizualizacijoms
